# Reconhecimento Facial Aplicado a um Sistema de Login

Este software tem o objetivo de solucionar o problema do login de usuário pela abordagem de reconhecimento facial.

O Reconhecimento facial se depara comumente com dois problemas:

- **Verificação Facial** - "esta é a pessoa reinvindicada?". Por exemplo, num sistema onde uma pessoa precisa usar um cartão de acesso em uma catraca, uma câmera apontada para o rosto da pessoa pode verificar se a pessoa que está usando o cartão é a pessoa correta (o dono do cartão de acesso). Um celular que desbloqueia usando seu rosto também está usando a verificação facial. Este é um problema de correspondência 1:1. 
- **Reconhecimento Facial** - "quem é essa pessoa?" Por exemplo, ao invés de usar um cartão de acesso, uma pessoa só precisaria ter sua face filmada por uma câmera, o sistema então compara a imagem captada com as imagens de rostos registradas na base de dados, se a pessoa existir na base, será reconhecida e poderá passar pela catraca. Este é um problema de correspondência de 1:K.

Usamos uma Rede Neural que codifica a imagem de um rosto em um vetor de 128 números. Comparando tais vetores, pode-se determinar se duas imagens correspondem à mesma pessoa.
    
**No desenvolvimento desta aplicação foram abordados: **
- A implementação da função de erro tripla
- Foi usado um modelo pré-treinado de rede neural para mapear imagens em um vetor codificado de 128 posições
- Essas codificações são então usadas para realizar a verificação facial e o reconhecimento facial

O código abaixo carrega as bibliotecas necessárias para o projeto. A maioria são modulos do framework de Machine Learning Keras e bibliotecas úteis da linguagem Python.

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
# Configuração necessária para o Jupyter exibir os gráficos gerados pela biblioteca matplotlib.
%matplotlib inline    
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

## 0 - Verificação Facial Ingênua

Na Verificação Facial, são dadas duas imagens e o sistema deve dizer se elas são da mesma pessoa. A forma mais fácil de fazer isso seria comparar as duas imagens pixel-por-pixel. Se a distância entre as imagens brutas for menor que um limite escolhido, pode ser a mesma pessoa!


É claro que esta abordagem é muito pobre, visto que os valores dos pixels podem mudar bastante devido à variações na iluminação, orientação do rosto da pessoa, e até mesmo pequenas mudanças na posição da cabeça e etc...

Por isso, melhor que usar imagens brutas, é possível aprender uma com uma função de codificação $f(img)$ para que as comparações de elementos dessa codificação forneçam avaliações mais precisas sobre se duas imagens são da mesma pessoa.

## 1 - Codificando imagens de rostos em um vetor de 128 posições

### 1.1 - Usando uma ConvNet para calcular codificações

O modelo de rede neural usa muitos dados e muito tempo para treinar. 

Como dissemos anteriormente, usamos um modelo de rede neural pré-treinado. Nós apenas precisamos carregar os parãmetros de peso treinados pelo modelo.


Mas é interessnate saber que:

- Esta rede recebe imagens RGB de 96x96 dimensões em sua entrada. Especificamente, a entrada é um rosto (ou um conjunto de imagens de rostos) como um tensor de dimensões $(m, n_C, n_H, n_W) = (m, 3, 96, 96)$ onde 'm' é o número de exemplos de treino, o '3' corresponde às três camadas RGB e 96x96 é a dimensão das imagens.
- Sua saída é uma matriz de $(m, 128)$ que codifica cada imagem de rosto de entrada em um vetor de 128 posições.

O código a seguir cria uma instância do modelo para imagens de rostos.

In [6]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [7]:
print("Quantidade total de parâmetros:", FRmodel.count_params())

Quantidade total de parâmetros: 3743280


Usando uma camada de 128 neurônios completamente conectados como sua última camada, o modelo garante que a saída será um vetor codificado de tamanho 128. Então essa codificação pode ser usada para comparar duas imagens de rostos.
Para uma codificação ser boa é preciso que: 
- A codificações de duas imagens da mesma pessoa sejam muito parecidas uma com a outra. 
- As codificações de duas imagens de diferentes pessoas sejam muito diferentes.

A função de erro tripla formaliza isso, e tenta "empurrar" as codificações das duas imagens da mesma pessoa (Âncora e Positiva) o mais pŕoximo possivel uma da outra, enquanto "puxa" as codificaçõe de imagens de diferentes pessoas (Âncora e Negativa) para mais lonje uma da outra.



### 1.2 - A Função de Erro Tripla

Para uma imagem $x$, denotamos sua codificação $f(x)$, onde $f$ é a função computada pela rede neural.


O treino usa triplas de imagens $(A, P, N)$ onde:  

- A é uma imagem "Âncora" -- a imagem de uma pessoa. 
- P é uma imagem "Positiva" -- uma imagem da mesma pessoa da imagem âncora.
- N é uma imagem "Negativa" -- uma imagem de uma pessoa diferente da pessoa da imagem âncora.

Essas triplas vêm do nosso conjunto de dados de treino. Nós escrevemos $(A^{(i)}, P^{(i)}, N^{(i)})$ para denotar o $i$-ésimo treino de exemplo. 

O Gradiente Descendente deve minimizar a seguinte função de erro tripla:

$$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ $$

Aqui, usa-se a notação "$[z]_+$" para definir $max(z,0)$.  

deve-se notar que:
- O primeiro termo (1) é a distância quadrada entre a âncora "A" e o a imagem positiva "P" para uma dada tripla; para que as imagens sejam iguais essas distância deve ser pequena. 
- O segundo termo (2) é a distância quadrada entre uma âncora "A" e a imagem negativa "N" para uma dada tripla, essa distância deve ser relativamente grande. 
- $\alpha$ é a margem. É um hiperparâmetro escolhido manualmente. Usamos $\alpha = 0.2$. 

O código a seguir define a função de erro tripla:

In [8]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementação da função de erro tripla.
    
    Argumentos:
    y_true -- rótulos true (verdadeiros), necessários quando se define uma perda em Keras, não é necessário nesta função.
    y_pred -- lista Python contendo três objetos:
            âncora -- as codificações para uma imagem de âncora, com as dimensões (None, 128)
            positiva -- as codificações para imagens positivas, com as dimensões (None, 128)
            negativa -- as codificações para imagens negativas, com as dimensões (None, 128)
    
    Returns:
    loss -- número real, valor de perda (erro)
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # Computa a distância entre a âncora e a positiva
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)

    # Computa a distância entre a âncora e a positiva
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    
    # subtrai as duas distãncias anteriores e adiciona o alpha
    basic_loss = pos_dist - neg_dist + alpha

    # Pega o máximo entre basic_loss e 0.0
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    
    return loss

In [9]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("Erro = " + str(loss.eval()))

Erro = 528.1427


## 2 - Carregando o modelo de treinado

No código seguinte nós carregamos um modelo previamente treinado. 

In [10]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

Na linha acima, foi carregado um modelo de rede neural que usa o algoritmo de otimização Adam, ele é uma versão melhorada do gradiente descendente, sua implementação é por conta do framework Tensorflow. Também foi passada como parâmetro a função triplet_loss que definimos anteriormente.

## 3 - Aplicando o modelo

Um dos problemas que se pode resolver com este modelo, é o da Verificação facial. Por exemplo: uma pessoa com um cartão de acesso tentando passar por uma catraca para ter acesso a algum prédio. A câmera capta a imagem do rosto da pessoa e verifica se essa pessoa que está usando o carão é a pessoa correta, ou seja, sem a necessidade de um ser humano para checar isso.

### 3.1 - Verificação Facial

O código a seguir funciona como uma base de dados simples contendo um vetor codificado para cada pessoa com permissão para entrar (passar pela catraca). Para gerar essa cofificação foi usado `img_to_encoding(image_path, model)` que basicamente executa o algoritmo forward propagation do modelo na imagem específica.

Nossa base de dados é representada por um dicionário Python. Este dicionário mapea cada nome de pessoa para um vetor codificado de 128 posições, ou seja para cada chave "nome" o valor é um vetor codificado de uma imagem.

É importante lembrar também que as imagens são todas da mesma dimensão, 96x96 pixels.

In [11]:
database = {}
database["patterson"] = img_to_encoding("images/patterson.jpg", FRmodel)
database["marcelo"] = img_to_encoding("images/marcelo.jpg", FRmodel)
database["alexandre"] = img_to_encoding("images/alexandre.jpg", FRmodel)

Na sequência é implementada a função responsável por verificar se a imagem captada por uma câmera é realmente a pessoa dona do cartão de acesso.Para isso foram implementados os seguintes passos computamos a codificação da imagem de entrada (a imagem captada pela câmera), em seguda calculamos a distância entre essa codificação e a codificação da imagem de identidade (a imagem do dono do cartão de acesso), por último informamos se a pessoa pode ou não passar, isso se a distância for menor que 0.7 (70%), do contrário o acesso é negado.

In [12]:
def verify(image_path, identity, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".
    
    Arguments:
    image_path -- path to an image
    identity -- string, name of the person you'd like to verify the identity. Has to be a resident of the Happy house.
    database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
    model -- your Inception model instance in Keras
    
    Returns:
    dist -- distance between the image_path and the image of "identity" in the database.
    door_open -- True, if the door should open. False otherwise.
    """

    # Computa a codificação da imagem
    encoding = img_to_encoding(image_path, model)
    
    # Computa a distância entre a imagem da câmera e a imagem da base
    dist = np.linalg.norm(encoding - database[identity])
    
    # Libera o acesso se a distância for menor que 0.7, do contrário nega o acesso. 
    if dist < 0.7:
        print("Olá " + str(identity) + ", bem vindo!")
        door_open = True
    else:
        print("Você não é " + str(identity) + ", acesso negado.")
        door_open = False
        
    return dist, door_open

No exemplo, Patterson está tentando passar pela catraca a câmera captura sua imagem ("images/camera_1.jpg"). O sistema deve decidir se ele pode ou não passar:

<img src="images/camera_1.jpg" style="width:100px;height:100px;">

A função verify() criada anteriormente verifica se a imagem captada corresponde ao verdadeiro dono do cartão, ou seja, se é Patterson usando seu cartão de acesso correto.

In [13]:
verify("images/camera_1.jpg", "patterson", database, FRmodel)

Olá patterson, bem vindo!


(0.63389784, True)

Marcelo perdeu seu cartão de acesso e está tentando entrar com o cartão de outra pessoa. A câmera capta sua imagem e a função de verificação não permite seu acesso. 

<img src="images/marcelo.jpg" style="width:100px;height:100px;">

In [14]:
verify("images/camera_1.jpg", "alexandre", database, FRmodel)

Você não é alexandre, acesso negado.


(0.7964067, False)

### 3.2 - Reconhecimento Facial

No Reconhecimento Facial queremos que o sistema reconheça uma pessoa apenas capturando uma imagem do seu rosto, para isso é preciso comparar a codificação da imagem capturada com as codificações das imagens das pessoas cadastradas na base. Se uma das imagens tiver uma distância menor ou igual a 0.7, então a pessoa da imagem capturada existe na base e portanto tem acesso liberado.

O código a seguir define uma função que faz esse trabalho. Primeiro é computada a codificação da imagem capturada, em seguida aa função procura qual das imagens codificadas da base possui a menor distância comparada à codificação da imagem capturada.

In [15]:
def who_is_it(image_path, database, model):
    """
    Implementa o Reconhecimento Facial.
    
    Argumentos:
    image_path -- caminho para a imagem no disco
    database -- base de dados contendo codificações de imagem, juntamente com o nome da pessoa na imagem
    model -- instancia do modelo de rede neural em Keras
    
    Returns:
    min_dist -- a distância mínima entre a codificação da image_path e as codificações encodings da base
    identity -- string, o nome predizido para a pessoa em image_path
    """
    
    # Codifica a imagem capturada
    encoding = img_to_encoding(image_path, model)
    
    # Inicializa a menor distância "min_dist" com um valor muito grande, 100.
    min_dist = 100
    
    # Encontra a imagem com a codificação mais próxima da imagem capturada
    # Itera sobre o dicionário da base de dados obtendo as chaves (names) e as codificações (db_enc).
    for (name, db_enc) in database.items():
        
        # Computa a distância entre a codificação da imagem capturada e a codificação acorrente da base de dados
        dist = np.linalg.norm(encoding - db_enc)

        # Se a distância for menor que a distÂncia mínima, então passa a ser a nova distância mínima, 
        # e a identidade passa a ser o nome corrente
        if dist < min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.7:
        print("Não existe na base de dados.")
    else:
        print ("Olá " + str(identity) + ", a distância é " + str(min_dist))
        
    return min_dist, identity

No código abaixo a função who_is_it() verifica se Patterson está na base de dados. 

In [16]:
who_is_it("images/camera_3.jpg", database, FRmodel)

Não existe na base de dados.


(0.9037236, 'patterson')